# T1567.002 Exfiltration to Cloud Storage

-----------------------------------------------------------------------

## Technique Description

Adversaries may exfiltrate data to a cloud storage service rather than over their primary command and control channel. Cloud storage services allow for the storage, edit, and retrieval of data from a remote cloud storage server over the Internet.

Examples of cloud storage services include Dropbox and Google Docs. Exfiltration to these cloud storage services can provide a significant amount of cover to the adversary if hosts within the network are already communicating with the service. 

## Technique Detection

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server) to known cloud storage services. Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. User behavior monitoring may help to detect abnormal patterns of activity.

-----------------------------------------------------------------------

### Tactics:

  *   Exfiltration

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Network Traffic:** Network Traffic Flow

  * **File:** File Access

  * **Command:** Command Execution

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Confucius | [Confucius](https://attack.mitre.org/groups/G0142) has exfiltrated victim data to cloud storage service accounts.(Citation: TrendMicro Confucius APT Feb 2018)| 
| ZIRCONIUM | [ZIRCONIUM](https://attack.mitre.org/groups/G0128) has exfiltrated stolen data to Dropbox.(Citation: Zscaler APT31 Covid-19 October 2020)| 
| HAFNIUM | [HAFNIUM](https://attack.mitre.org/groups/G0125) has exfiltrated data to file sharing sites, including MEGA.(Citation: Microsoft HAFNIUM March 2020)| 
| Chimera | [Chimera](https://attack.mitre.org/groups/G0114) has exfiltrated stolen data to OneDrive accounts.(Citation: NCC Group Chimera January 2021)| 
| Kimsuky | [Kimsuky](https://attack.mitre.org/groups/G0094) has exfiltrated stolen files and data to actor-controlled Blogspot accounts.(Citation: Talos Kimsuky Nov 2021)| 
| Leviathan | [Leviathan](https://attack.mitre.org/groups/G0065) has used an uploader known as LUNCHMONEY that can exfiltrate files to Dropbox.(Citation: Proofpoint Leviathan Oct 2017)(Citation: FireEye Periscope March 2018)| 
| FIN7 | [FIN7](https://attack.mitre.org/groups/G0046) has exfiltrated stolen data to the MEGA file sharing site.(Citation: CrowdStrike Carbon Spider August 2021)| 
| Lazarus Group | [Lazarus Group](https://attack.mitre.org/groups/G0032) has exfiltrated stolen data to Dropbox using a customized version of dbxcli.(Citation: ESET Lazarus Jun 2020)(Citation: ClearSky Lazarus Aug 2020)| 
| Threat Group-3390 | [Threat Group-3390](https://attack.mitre.org/groups/G0027) has exfiltrated stolen data to Dropbox.(Citation: Trend Micro DRBControl February 2020)| 
| Turla | [Turla](https://attack.mitre.org/groups/G0010) has used WebDAV to upload stolen USB files to a cloud drive.(Citation: Symantec Waterbug Jun 2019) [Turla](https://attack.mitre.org/groups/G0010) has also exfiltrated stolen files to OneDrive and 4shared.(Citation: ESET ComRAT May 2020)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1567/002)

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 20 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, Servando Quinones

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Turla will compress files and exfiltrate them to a cloud storage provider such as OneDrive or 4shared.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Encrypted data may make analyzing the packet payload difficult. Use connection metadata to try and gather as much information as possible about the connection.
- Incorrect sensor placement will make this TTP difficult to detect if the required traffic is being missed.

## Analytical References

  * https://www.welivesecurity.com/wp-content/uploads/2020/05/ESET_Turla_ComRAT.pdf

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Commercial cloud storage can exist on network without being malicious.
- If commercial cloud is mounted as a share with the ‘net use’ command, it is an indicator of Turla TTPs.
- There are more ways to mount a share than what is discussed here (powershell, wmic, etc...).
- Monitor for other cloud providers that may be seen across the network (OndeDrive, Google, Dropbox, Pastbin style sites)

#### Analytic 1

  * **Information:** Identify net use command executed.

  * **Source:** Security Logs, Winlogbeats, Sysmon

  * **Tool:** Kibana

  * **Notes:** 1 (Sysmon) & 4688 (Win Audit) = Process Creation.

  * **Query:** ```(Event_ID:4688 OR Event_ID: 1) AND "net*.exe"```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Commercial cloud storage can exist on network without being malicious.
- If commercial cloud is mounted as a share with the ‘net use’ command, it is an indicator of Turla TTPs.
- There are more ways to mount a share than what is discussed here (powershell, wmic, etc...).
- Monitor for other cloud providers that may be seen across the network (OndeDrive, Google, Dropbox, Pastbin style sites)

#### Analytic 1

  * **Information:** Detect large uncommon data flows to known cloud services (WebDAV | SMB)

  * **Source:** PCAP

  * **Tool:** Arkime

  * **Notes:** Web Distributed Authoring and Versioning (WebDAV) is an extension to Hypertext Transfer Protocol (HTTP) that

defines how basic file functions such as copy, move, delete, and create are performed by using HTTP. Sort by bytes.

  * **Query:** ```ip.dst != internal && protocols == [smb, tls]```
  * **Query:** ```http.uri == *webdav*```

#### Analytic 2

  * **Information:** Detect possible data exfiltration over SMB.

  * **Source:** PCAP

  * **Tool:** Arkime

  * **Notes:** N/A

  * **Query:** ```protocols == [smb, rpc] && ip.src == [10/8, 172.16/12, 192.168/16] && ip.dst != [10/8, 172.16/12, 192.168/16]```

#### Analytic 3

  * **Information:** Identify possible cloud storage providers that may be seen across the network and verify use with MP.

  * **Source:** PCAP

  * **Tool:** Arkime

  * **Notes:** Common cloud storage providers like OneDrive, Google drive with a combination of SMB "net use" may need further analysis. Use of http entropy maybe used, entropy.http == <range> [0-8]

  * **Query:** ```http.uri == [*onedrive*, *4shared*, *drive.google*, *docs.live*] ```
  * **Query:** ```protocols == [smb, dcerpc] && dcerpc.cmd == *NetUse*```
